In [175]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from bs4 import BeautifulSoup
import pandas as pd

def start(hl=True):
    #Simulated browser login
    fireFoxOptions = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=fireFoxOptions)
#     driver.get("https://www.kingsofchaos.com")
    time.sleep(4)
    #LOGIN
    return driver

In [176]:
d = start()

In [177]:
urls_to_do = ['https://dharmatreasure.org/how-to-deal-with-wrongs-weve-done/']
urls_done = []
articles = []

for i in range(1,8):
    d.get('https://dharmatreasure.org/section/dharma-talks/page/{}'.format(i))
    for l in d.find_elements_by_tag_name('a'):
        if l.get_attribute('rel') != '':
            url = l.get_attribute('href')
            if 'wp-login' not in url:
                urls_to_do.append(url)

In [178]:
while len(urls_to_do) > 0:
    url = urls_to_do.pop()
    if url in urls_done:
        continue
    d.get(url)
    html = BeautifulSoup(d.page_source)
    art = html.find('article')
    try:
        audio = art.find('source')['src']
    except TypeError:
        audio = None
    except AttributeError:
        urls_done.append(url)
        continue
    try:
        articles.append({
            'article': str(art),
            'url': url,
            'title': art.find('header').text.strip(),
            'content': art.find('div').text.strip().split('http://s3.amazonaws.com/dharmatreasur')[0].strip(),
            'audio_link': audio
        })
    except AttributeError:
        urls_done.append(url)
        continue
    urls_done.append(url)
    pd.DataFrame(articles).to_json('./articles-selenium.json')
    # find next links
    for link in html.find_all('a', href=True):
        try:
            if link['title'] == 'Previous' or link['title'] == 'Next':
                urls_to_do.append(link['href'])
        except KeyError:
            continue